<a href="https://colab.research.google.com/github/EwFox76/IMERS-O-ALURA-IA/blob/main/Agende_de_suporte_emocional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar bibliotecas necessárias
!pip install -q google-genai ipywidgets google-auth google-api-python-client google-auth-oauthlib google-auth-httplib2 google-auth-httplib2

# Importar bibliotecas
import os
import pickle
from datetime import datetime

from google.colab import userdata
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display

from google import genai
from google.genai import types

# Configurar API Key (substitua pela sua chave correta)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Inicializar cliente Gemini 2.5 flash preview
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Configurar sistema do chat
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto, dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
É FUNDAMENTAL que você NUNCA aja como um profissional de saúde mental.
Suas respostas devem ser:
- Empáticas e acolhedoras.
- Focadas em positividade e encorajamento.
- Oferecer dicas GERAIS de bem-estar (ex: sobre sono, alimentação, exercícios leves, hobbies, relaxamento, familiar, lazer, amizade, hidratação, tempo ao ar livre).
- NUNCA fazer diagnósticos ou sugerir tratamentos ou medicamentos.
- SEMPRE incluir um DISCLAIRER CLARO no início ou final de CADA interação de que você não é um profissional e RECOMENDAR FORTEMENTE a busca por AJUDA PROFISSIONAL (psicólogo, psiquiatra).
- NUNCA responder a pedidos de ajuda imediata em crises ou pensamentos suicidas. Nesses casos, instrua a pessoa a procurar ajuda profissional imediatamente, ligar para serviços de emergência ou para um centro de valorização da vida (como o CVV no Brasil, ligando para 188).

Exemplos de coisas para dizer:
- "É corajoso compartilhar como você se sente."
- "Lembre-se de ser gentil consigo mesmo."
- "Pequenos passos fazem diferença."
- "Que tal tentar uma caminhada curta hoje, se possível?"
- "Respirar fundo por alguns minutos pode ajudar a relaxar."
- "Lembre-se que você não está sozinho(a)."
- "Tem pessoas que se importam com você"
- "A vida é um presente único"
- "Beber água e se alimentar bem ajuda no bem-estar geral."

Exemplos de coisas para NÃO dizer:
- "Você tem depressão." (Diagnóstico)
- "Tome este medicamento." (Tratamento)
- "Eu posso curar sua depressão." (Afirmação falsa)
- "Não se sinta assim." (Invalidando sentimentos)
- "É só uma fase, vai passar." (Minimizando)

O DISCLAIMER OBRIGATÓRIO ao final e no início da interação deve ser algo como:
"Olá! Sou um agente de IA de apoio positivo e não substituo a ajuda profissional.
Se você está enfrentando dificuldades significativas, por favor, procure a ajuda de um"
OU
"Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
Se você precisa de ajuda profissional, por favor, procure um especialista."

OU ALGO SEMELHANTE, MAS CLARO E RECOMENDANDO AJUDA PROFISSIONAL.

Vamos começar. Apresente-se brevemente com o disclaimer inicial.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Substituto de YouTube: Links de músicas relaxantes por pesquisa Google (exemplo)
def buscar_musicas_reais(sentimento):
    # Pesquisa simplificada - aqui simulamos resultado, poderia ser integrado com busca real
    sugestoes = [
        "🎵 Calm Vibes Radio: https://calmvibesradio.com/",
        "🎵 Relaxing Music Online: https://www.relaxingmusic.com/",
        "🎵 Free Music Archive - Ambient: https://freemusicarchive.org/genre/Ambient/"
    ]
    texto = f"🔕 Não consegui buscar no YouTube, mas aqui estão sugestões de sites com músicas relaxantes para você:\n" + "\n".join(sugestoes)
    return texto

# Widgets e variáveis globais
chat_active = True
user_name = ""
output_area = Output(layout={'border': '2px solid black', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='100px'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Sair da Conversa', button_style='danger')

# Função para buscar frases inspiradoras via Gemini
def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas e serenas de conforto, inspiração e motivação com emojis."
    try:
        resposta = chat.send_message(prompt)
        return resposta.text if resposta else "Vamos respirar fundo e seguir em frente. 🌿"
    except Exception:
        return "Vamos respirar fundo e seguir em frente. 🌿"

# Função para imprimir no output sem erro
def imprimir(texto):
    with output_area:
        print(texto)

# Função para mostrar eventos do Google Calendar (opcional)
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

def conectar_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_console()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def mostrar_eventos_do_dia():
    try:
        service = conectar_google_calendar()
        now = datetime.utcnow().isoformat() + 'Z'
        eventos = service.events().list(calendarId='primary', timeMin=now,
                                        maxResults=5, singleEvents=True, orderBy='startTime').execute()
        itens = eventos.get('items', [])
        if not itens:
            return "Nenhum evento encontrado."
        resposta = "\n📅 Eventos do dia:\n"
        for evento in itens:
            inicio = evento['start'].get('dateTime', evento['start'].get('date'))
            resposta += f"\n🕒 {inicio} - {evento['summary']}"
        return resposta
    except Exception:
        return "Não foi possível acessar sua agenda."

# Lógica do botão enviar
def on_send_button_clicked(b):
    global chat_active, user_name
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''
        if not user_name:
            user_name = user_input
            imprimir(f"Olá, {user_name}! 🌟 Como você está se sentindo hoje? 😊\n")
            return
        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return

        imprimir(f"{user_name}: {user_input}\n")

        # Enviar mensagem para Gemini e mostrar resposta
        try:
            response = chat.send_message(user_input)
            imprimir(f"Resposta: {response.text}\n")
        except Exception:
            imprimir("⚠️ Não consegui responder agora. Tente novamente.\n")

        # Frases motivacionais
        frases = buscar_frases_inspiradoras()
        imprimir(f"✨ Frases para você:\n{frases}\n")

        # Sugestão de músicas para estados emocionais específicos
        if any(emocao in user_input.lower() for emocao in ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]):
            musica = buscar_musicas_reais(user_input.lower())
            imprimir(f"🎵 Músicas sugeridas:\n{musica}\n")

        # Mostrar agenda, se solicitado
        if "agenda" in user_input.lower():
            eventos = mostrar_eventos_do_dia()
            imprimir(eventos)

# Botão sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    imprimir("Conversa encerrada. Obrigado por compartilhar! 🌸")

# Conectar botões aos eventos
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Montar e exibir interface
display(output_area, input_area, HBox([send_button, exit_button]))
imprimir("Olá! 😊 Sou seu agente de bem-estar virtual. Qual é o seu nome?")
